In [ ]:
# =============================================================================
# COLAB 4: GRPO (Group Relative Policy Optimization) — REASONING MODEL
# - Uses TRL's GRPOTrainer with a real reward_fn (math accuracy)
# - Hard token budgets (prompt+completion) to avoid >max_seq issues
# - LoRA r=32 on attention+MLP; 4-bit loading; W&B off by default
# =============================================================================

# Cell 1: Install deps
# -----------------------------------------------------------------------------
print("📦 Installing Unsloth + TRL (GRPO) ...")
!pip install -q unsloth bitsandbytes accelerate datasets transformers
!pip install -q "trl>=0.15.0"   # GRPO is available in recent TRL

print("✅ Installation complete!")

# Cell 2: Imports & env
# -----------------------------------------------------------------------------
import os, re, math, statistics, torch
from datasets import Dataset
from unsloth import FastLanguageModel
from transformers import set_seed
from trl import GRPOConfig, GRPOTrainer

# Disable Weights & Biases by default
os.environ["WANDB_DISABLED"] = "true"
# If you hit odd TorchDynamo traces, uncomment:
# os.environ["TORCHDYNAMO_DISABLE"] = "1"

print(f"🔥 PyTorch: {torch.__version__}")
print(f"🎮 CUDA: {torch.cuda.is_available()} | GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

# Cell 3: Config
# -----------------------------------------------------------------------------
# Overall context
max_seq_length       = 1024     # total context window for model
max_prompt_length    = 512      # GRPO will enforce budgets
max_completion_length= 384      # 512+384=896 <= 1024 (headroom for specials)

# Model load
dtype         = None
load_in_4bit  = True

# LoRA
lora_r        = 32
lora_alpha    = 32
lora_dropout  = 0.05

# GRPO params
num_generations = 4     # per prompt
temperature     = 0.8
beta_kl         = 0.0   # KL off by default; set >0 to enable
loss_type       = "dapo"  # length-bias resistant (also try "dr_grpo")

# Train params
batch_size   = 1
grad_accum   = 8
num_epochs   = 1
lr           = 5e-6
max_steps    = 50
seed         = 3407
set_seed(seed)

print(f"""
🔧 Config:
 • LoRA r/α/drop: {lora_r}/{lora_alpha}/{lora_dropout}
 • GRPO: G={num_generations}, T={temperature}, KL β={beta_kl}, loss='{loss_type}'
 • Budgets: prompt≤{max_prompt_length}, completion≤{max_completion_length}, ctx≤{max_seq_length}
 • Train: bs={batch_size}, accum={grad_accum}, steps={max_steps}, lr={lr}
""")

# Cell 4: Load base model (Unsloth)
# -----------------------------------------------------------------------------
print("📥 Loading base model...")
model_name = "HuggingFaceTB/SmolLM2-135M-Instruct"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# Ensure tokenizer/pad/eos/truncation are explicit
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if getattr(model.config, "pad_token_id", None) is None:
    model.config.pad_token_id = tokenizer.eos_token_id
tokenizer.model_max_length = max_seq_length
tokenizer.truncation_side = "right"
tokenizer.padding_side = "right"

print("✅ Model loaded.")

# Cell 5: Apply LoRA
# -----------------------------------------------------------------------------
print("🔧 Applying LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_r,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",     # attention
        "gate_proj", "up_proj", "down_proj",        # MLP
    ],
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=seed,
    max_seq_length=max_seq_length,
)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in model.parameters())
print(f"✅ Trainable: {trainable_params/1e6:.2f}M / {total_params/1e6:.2f}M "
      f"({100*trainable_params/total_params:.2f}%)")

# Cell 6: Build small math dataset (prompt + answer)
# -----------------------------------------------------------------------------
print("📚 Building toy math dataset (for demo)...")
math_problems = [
    {
        "problem": "If a store has 45 apples and sells 17, then buys 23 more, how many apples does it have?",
        "answer": "51",
    },
    {
        "problem": "A rectangle has length 8 cm and width 5 cm. What is its perimeter?",
        "answer": "26",
    },
    {
        "problem": "If 3x + 5 = 20, what is x?",
        "answer": "5",
    },
    {
        "problem": "A train travels 120 km in 2 hours. What is its average speed?",
        "answer": "60",
    },
    {
        "problem": "If you have 3 bags with 4 apples each, and 2 bags with 6 apples each, how many apples total?",
        "answer": "24",
    },
] * 100  # 500 problems

def format_problem(p):
    return (
        "Solve this problem step by step.\n\n"
        f"Problem: {p}\n\n"
        "Let me think through this:"
    )

dataset = Dataset.from_list([
    {"prompt": format_problem(item["problem"]), "answer": item["answer"]}
    for item in math_problems
])

print(f"✅ Dataset size: {len(dataset)}")
print("📝 Sample prompt:\n", dataset[0]["prompt"][:200], "...")

# Cell 7: Reward function (math accuracy)
# -----------------------------------------------------------------------------
def extract_answer(text: str):
    """Heuristic extraction of final numeric answer."""
    patterns = [
        r'answer is (\d+)',
        r'answer:\s*(\d+)',
        r'=\s*(\d+)',
        r'equals\s+(\d+)',
        r'total is (\d+)',
    ]
    t = text.lower()
    for pat in patterns:
        m = re.search(pat, t)
        if m:
            return m.group(1)
    nums = re.findall(r'\d+', t)
    return nums[-1] if nums else None

def math_accuracy_reward(completions, **kwargs):
    """
    completions: list[list[{"content": str}]]
    kwargs may include: 'answer' from dataset (vectorized to a list)
    Returns list[float]: 1.0 if predicted == answer else 0.0
    """
    answers = kwargs.get("answer", None)
    rewards = []
    for i, comp in enumerate(completions):
        # TRL passes each completion as [{"content": "..."}]
        text = comp[0]["content"] if isinstance(comp, list) and comp else str(comp)
        pred = extract_answer(text)
        gold = answers[i] if isinstance(answers, list) else answers
        rewards.append(1.0 if (pred is not None and gold is not None and str(pred) == str(gold)) else 0.0)
    return rewards

print("✅ Reward function ready (exact numeric match).")

# Cell 8: GRPO config
# -----------------------------------------------------------------------------
print("⚙️ Building GRPOConfig...")
train_args = GRPOConfig(
    output_dir="./grpo_reasoning_smollm2",
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,
    warmup_steps=5,
    max_steps=max_steps,                       # takes precedence over epochs
    num_train_epochs=num_epochs,
    learning_rate=lr,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_bnb_8bit",                    # 4-bit friendly
    lr_scheduler_type="linear",
    seed=seed,
    save_strategy="steps",
    save_steps=25,
    report_to=[] if os.environ.get("WANDB_DISABLED","true").lower()=="true" else ["wandb"],

    # GRPO-specific
    num_generations=num_generations,
    temperature=temperature,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    beta=beta_kl,                              # KL term weight (0.0 by default)
    loss_type=loss_type,                       # "dapo" recommended; try "dr_grpo"
    scale_rewards="batch",                     # robust reward scaling
    mask_truncated_completions=True,           # ignore clipped completions in loss
)
print("✅ GRPOConfig ready.")

# Cell 9: Trainer
# -----------------------------------------------------------------------------
print("🏋️ Initializing GRPOTrainer...")
trainer = GRPOTrainer(
    model=model,                    # Unsloth model object works fine
    processing_class=tokenizer,     # tokenizer (aka processing_class)
    reward_funcs=math_accuracy_reward,
    args=train_args,
    train_dataset=dataset,          # expects 'prompt' column + extra cols (e.g., 'answer')
    # eval_dataset=...,             # (optional) add held-out split if you have one
)
print("✅ Trainer ready!")

# Cell 10: Train
# -----------------------------------------------------------------------------
print("🚀 Starting GRPO training ...")
print("="*60)
result = trainer.train()
metrics = result.metrics or {}
print("="*60)
print("✅ Training complete!")
print("📊 Stats:")
print("   • Steps:", metrics.get("train_steps", metrics.get("global_step", "N/A")))
print("   • Train loss:", metrics.get("train_loss", "N/A"))
print("   • Time (s):", metrics.get("train_runtime", "N/A"))

# Cell 11: Save (adapters + optional merged)
# -----------------------------------------------------------------------------
print("💾 Saving GRPO-trained adapters...")
model.save_pretrained("smollm2_grpo_adapters")
tokenizer.save_pretrained("smollm2_grpo_adapters")
print("✅ Adapters → ./smollm2_grpo_adapters")

print("\n🔧 Saving merged model (optional)...")
merged_ok = False
try:
    # If available in your Unsloth version:
    model.save_pretrained_merged(
        "smollm2_grpo_merged",
        tokenizer,
        save_method="merged_16bit",
    )
    merged_ok = True
except Exception as e:
    print("   save_pretrained_merged unavailable, trying manual merge:", repr(e))
    try:
        from unsloth import FastLanguageModel as _FLM
        _FLM.merge_lora_weights(model)
        model.save_pretrained("smollm2_grpo_merged")
        tokenizer.save_pretrained("smollm2_grpo_merged")
        merged_ok = True
    except Exception as e2:
        print("   Manual merge failed (not critical):", repr(e2))

print("✅ Merged → ./smollm2_grpo_merged" if merged_ok else "ℹ️ Skipping merge; adapters saved and usable.")

# Cell 12: Inference (reasoning demo)
# -----------------------------------------------------------------------------
print("\n🧪 Reasoning demo\n")
from torch import inference_mode
FastLanguageModel.for_inference(model)

def solve(problem_text, temp=0.3, max_new=256):
    prompt = (
        "Solve this problem step by step.\n\n"
        f"Problem: {problem_text}\n\n"
        "Let me think through this:"
    )
    inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(model.device)
    with inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new,
            temperature=temp,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):].strip() if text.startswith(prompt) else text

tests = [
    "A bakery made 96 cupcakes. They sold 37 in the morning and 28 in the afternoon. How many are left?",
    "If a book costs $12 and you buy 3 books, how much do you spend?",
    "A triangle has sides of length 5 cm, 12 cm, and 13 cm. Is it a right triangle?",
]
print("="*60)
print("REASONING TEST RESULTS")
print("="*60)
for i, q in enumerate(tests, 1):
    print(f"\n[Test {i}]")
    print("Problem:", q)
    print("Solution:\n", solve(q))
    print("-"*60)

# Cell 13: Summary
# -----------------------------------------------------------------------------
print("""
╔════════════════════════════════════════════════════════════╗
║            GRPO REASONING TRAINING — SUMMARY               ║
╚════════════════════════════════════════════════════════════╝
• Base: HuggingFaceTB/SmolLM2-135M-Instruct
• Method: GRPO (loss='dapo', β_KL=0.0), LoRA r=32 (attn+MLP)
• Data: 500 toy math problems (prompt+answer)
• Safety: budgets prompt≤512, completion≤384, total≤1024
• Saved:
    - Adapters: ./smollm2_grpo_adapters
    - Merged (optional): ./smollm2_grpo_merged
• Tips:
    - Try loss_type='dr_grpo' and different scale_rewards ('batch'|'none')
    - Increase num_generations (e.g., 8) for stronger exploration
    - Train longer and with real datasets (GSM8K, MATH) for better gains
""")


📦 Installing Unsloth + TRL (GRPO) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.7/348.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.6/273.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the pac

🦥 Unsloth Zoo will now patch everything to make training faster!
🔥 PyTorch: 2.8.0+cu126
🎮 CUDA: True | GPU: Tesla T4

🔧 Config:
 • LoRA r/α/drop: 32/32/0.05
 • GRPO: G=4, T=0.8, KL β=0.0, loss='dapo'
 • Budgets: prompt≤512, completion≤384, ctx≤1024
 • Train: bs=1, accum=8, steps=50, lr=5e-06

📥 Loading base model...
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

✅ Training complete!
📊 Stats:
   • Steps: N/A
   • Train loss: -9.685754776000977e-10
   • Time (s): 1833.6566
💾 Saving GRPO-trained adapters...
✅ Adapters → ./smollm2_grpo_adapters

🔧 Saving merged model (optional)...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `smollm2_grpo_merged`: 100%|██████████| 1/1 [00:04<00:00,  4.42s/it]


Successfully copied all 1 files from cache to `smollm2_grpo_merged`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:04<00:00,  4.06s/it]


Unsloth: Merge process complete. Saved to `/content/smollm2_grpo_merged`
✅ Merged → ./smollm2_grpo_merged

🧪 Reasoning demo

REASONING TEST RESULTS

[Test 1]
Problem: A bakery made 96 cupcakes. They sold 37 in the morning and 28 in the afternoon. How many are left?
Solution:
 First, I'll start by calculating the number of cupcakes sold in the morning. I'll find the number of cupcakes sold in the morning by subtracting the number of cups sold in the afternoon from the number of cups sold in the morning.

So, I'll find the number of cupcakes sold in the morning by subtracting the number of cups sold in the afternoon from the number of cups sold in the morning.

Now, I'll find the number of cupcakes sold in the afternoon by subtracting the number of cups sold in the afternoon from the number of cups sold in the morning.

So, I'll find the number of cupcakes sold in the afternoon by subtracting the number of cups sold in the afternoon from the number of cups sold in the morning.

Next, I'l